In [11]:
import os
import requests
from dotenv import load_dotenv

load_dotenv(".env")

API_URL = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {os.environ["OPENROUTER_API_KEY"]}",
    "Content-Type": "application/json",
}

RQ = """Task: Identify and correct code smells in the provided **Python** code snippet. If there is no discernible code smell, respond with 'None'. Then, suggest which modifications or refactorings could be applied to mitigate the identified code smell.

Instructions:  
1. Analyze the code for code smells (e.g., duplicated code, long methods, magic numbers, poor naming).  
2. For each smell, provide:  
   - **Type**: A tuple `(code_smell_type, start_line, end_line)`.  
   - **Correction**: The **entire rewritten code** with all fixes applied, marking new sections with `====== [ADDED CODE] =======`.  

3. **Output Format**:  

Code Smells Detected:
Type: (CodeSmellType, start_line, end_line)
Type: (CodeSmellType, start_line, end_line)
...

Corrected Code
====== [CORRECTED CODE START] =======
[Full corrected code here]
====== [CORRECTED CODE END] =======
"""

In [12]:
def make_question(request, snippet):
    return f"""{request}

{snippet}"""

def load_snippets(folder_path):
    snippets = []
    for root, _, files in os.walk(folder_path):
        py_files = sorted(f for f in files if f.endswith(".py"))
        for filename in py_files:
            file_path = os.path.join(root, filename)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    snippets.append((filename, content))
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return snippets


def save_markdown(filename, content):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Successfully saved content to '{filename}'")
    except Exception as e:
        print(f"Error saving to file: {e}")

In [13]:
def process_snippets(folder, request=RQ, snippets=[]):
    """Process code snippets with the given request."""
    print(snippets)
    for filename, snippet in snippets:
        try:
            data = {
                "model": "deepseek/deepseek-chat:free",
                "messages": [{"role": "user", "content": make_question(request, snippet)}],
            }

            response = requests.post(API_URL, json=data, headers=headers)
            print(f"Response status code: {response.status_code}")
            if response.status_code == 200:
                parsed_response = response.json()
                answer = """"""
                for index, resp in enumerate(parsed_response["choices"]):
                    content = resp["message"]["content"]
                    if index == 0:
                        answer += content
                    else:
                        answer += "\n\n" + content
                save_markdown(f"responses/{folder}/{filename}.md", answer)
            else:
                print(
                    "Falha ao buscar dados da API do OpenRouter. Código de Status:",
                    response.status_code,
                )
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")

In [ ]:
snippets_folder = "../pyparsing/pyparsing"
snippets = load_snippets(snippets_folder)
request = RQ

process_snippets(request=request, snippets=snippets)

In [ ]:
snippets_folder = "../requests/src/requests"
snippets = load_snippets(snippets_folder)
request = RQ

process_snippets("requests", request=request, snippets=snippets)

In [14]:
# # de codesmells adicionados: 20
# Arquivos: __main.py__, _completion_classes.py, cli.py, colors.py, completion.py, core.py, models.py,
#           params.py, rich_utils.py, testing.py

snippets_folder = "../typer/typer"
snippets = load_snippets(snippets_folder)
request = RQ

process_snippets("typer", request=request, snippets=snippets)

[('__init__.py', '"""Typer, build great CLIs. Easy to code. Based on Python type hints."""\n\n__version__ = "0.16.0"\n\nfrom shutil import get_terminal_size as get_terminal_size\n\nfrom click.exceptions import Abort as Abort\nfrom click.exceptions import BadParameter as BadParameter\nfrom click.exceptions import Exit as Exit\nfrom click.termui import clear as clear\nfrom click.termui import confirm as confirm\nfrom click.termui import echo_via_pager as echo_via_pager\nfrom click.termui import edit as edit\nfrom click.termui import getchar as getchar\nfrom click.termui import pause as pause\nfrom click.termui import progressbar as progressbar\nfrom click.termui import prompt as prompt\nfrom click.termui import secho as secho\nfrom click.termui import style as style\nfrom click.termui import unstyle as unstyle\nfrom click.utils import echo as echo\nfrom click.utils import format_filename as format_filename\nfrom click.utils import get_app_dir as get_app_dir\nfrom click.utils import get_b